# 1.Resnet

In [1]:
import torch
from torch import nn,optim
from torch.nn import functional as F

In [5]:
class ResBlk(nn.Module):
    def __init__(self,ch_in,ch_out):
        super(ResBlk,self).__init__()
        self.conv1 = nn.Conv2d(ch_in,ch_out,kernel_size=3,stride=1,padding=1)
        self.bn1 = nn.BatchNorm2d(ch_out)
        self.conv2 = nn.Conv2d(ch_out,ch_out,kernel_size=3,stride=1,padding=1)
        self.bn = nn.BatchNorm2d(ch_out)
        self.extra = nn.Sequential()
        if ch_out!= ch_in:
            self.extra = nn.Sequential(
            nn.Conv1d(ch_in,ch_out,kernel_size=1,stride = 1),
            nn.BatchNorm2d(ch_out)
            )
    def forward(self,x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.conv2(out)
        out = self.bn(out)
        out = self.extra(x+out)
        return out

# 2.模型内部参数管理
* 可以用.parameters()或者.named_parameters()返回其内的所有参数的迭代器

In [6]:
net = nn.Sequential(
    nn.Linear(2,4),
    nn.Linear(4,1)
)

In [7]:
list(net.parameters())

[Parameter containing:
 tensor([[-0.2172, -0.4014],
         [ 0.6063, -0.0481],
         [ 0.6055, -0.6054],
         [-0.6814, -0.5784]], requires_grad=True), Parameter containing:
 tensor([-0.5971, -0.6531,  0.5061,  0.2764], requires_grad=True), Parameter containing:
 tensor([[-0.4300, -0.2249,  0.2508, -0.3570]], requires_grad=True), Parameter containing:
 tensor([-0.0399], requires_grad=True)]

In [8]:
dict(net.named_parameters())

{'0.weight': Parameter containing:
 tensor([[-0.2172, -0.4014],
         [ 0.6063, -0.0481],
         [ 0.6055, -0.6054],
         [-0.6814, -0.5784]], requires_grad=True),
 '0.bias': Parameter containing:
 tensor([-0.5971, -0.6531,  0.5061,  0.2764], requires_grad=True),
 '1.weight': Parameter containing:
 tensor([[-0.4300, -0.2249,  0.2508, -0.3570]], requires_grad=True),
 '1.bias': Parameter containing:
 tensor([-0.0399], requires_grad=True)}

# 3.模型树形结构
* 模块之间通过嵌套组合会形成树形结构,使用.children()可以获取其直接孩子节点
* 使用.modules()可以直接获取其所有子节点

In [11]:
class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet,self).__init__()
        self.net = nn.Linear(4,3)
    def forward(x):
        return self.net(x)
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet,self).__init__()
        self.net = nn.Sequential(
        BaseNet(),
        nn.ReLU(inplace = True),
        nn.Linear(3,2)
        )
    def forward(x):
        x = self.net(x)
        return x

In [12]:
mynet = MyNet()

In [13]:
list(mynet.children())

[Sequential(
   (0): BaseNet(
     (net): Linear(in_features=4, out_features=3, bias=True)
   )
   (1): ReLU(inplace=True)
   (2): Linear(in_features=3, out_features=2, bias=True)
 )]

In [14]:
list(mynet.modules())

[MyNet(
   (net): Sequential(
     (0): BaseNet(
       (net): Linear(in_features=4, out_features=3, bias=True)
     )
     (1): ReLU(inplace=True)
     (2): Linear(in_features=3, out_features=2, bias=True)
   )
 ), Sequential(
   (0): BaseNet(
     (net): Linear(in_features=4, out_features=3, bias=True)
   )
   (1): ReLU(inplace=True)
   (2): Linear(in_features=3, out_features=2, bias=True)
 ), BaseNet(
   (net): Linear(in_features=4, out_features=3, bias=True)
 ), Linear(in_features=4, out_features=3, bias=True), ReLU(inplace=True), Linear(in_features=3, out_features=2, bias=True)]

# 4.加载和保存
* net.state_dict()传入torch.save()保存模型
* torch.load()加载参数后传入net.load_state_dict()

In [17]:
torch.save(mynet.state_dict(),'ckpt.mdl')
mynet.load_state_dict(torch.load('ckpt.mdl'))

<All keys matched successfully>

# 4.own linear layer
# nn.Parameter和.parameters()
* 如果在继承nn.Module类来实现模块时，出现需要操作Tensor的部分

In [19]:
class MyLinear(nn.Module):
    def __init__(self,inp,oup):
        super(MyLinear,self).__init__()
        
        #requires_grad = True
        self.w = nn.Parameter(torch.randn(oup,inp))
        self.b = nn.Parameter(torch.randn(oup))
        
    def forward(self,x):
        x = x@w.t()+self.b
        return x